<a href="https://colab.research.google.com/github/sarthak-2002/PySpark_Learning/blob/main/Introduction_to_ML_using_Pyspark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyspark

     |████████████████████████████████| 212.4MB 74kB/s 
     |████████████████████████████████| 204kB 18.5MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.2-py2.py3-none-any.whl size=212880768 sha256=b3e660dbcc0c4edb9406007abf6dfa938cffcbb7ee2cc3f97fcac6e033c204d3
  Stored in directory: /root/.cache/pip/wheels/40/1b/2c/30f43be2627857ab80062bef1527c0128f7b4070b6b2d02139
Successfully built pyspark


In [2]:
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Dataframe').getOrCreate()
spark

In [3]:
df_tips=spark.read.csv("tips.csv",header=True, inferSchema = True)
df_tips.show()

+----------+----+------+------+---+------+----+
|total_bill| tip|   sex|smoker|day|  time|size|
+----------+----+------+------+---+------+----+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|
|     25.29|4.71|  Male|    No|Sun|Dinner|   4|
|      8.77| 2.0|  Male|    No|Sun|Dinner|   2|
|     26.88|3.12|  Male|    No|Sun|Dinner|   4|
|     15.04|1.96|  Male|    No|Sun|Dinner|   2|
|     14.78|3.23|  Male|    No|Sun|Dinner|   2|
|     10.27|1.71|  Male|    No|Sun|Dinner|   2|
|     35.26| 5.0|Female|    No|Sun|Dinner|   4|
|     15.42|1.57|  Male|    No|Sun|Dinner|   2|
|     18.43| 3.0|  Male|    No|Sun|Dinner|   4|
|     14.83|3.02|Female|    No|Sun|Dinner|   2|
|     21.58|3.92|  Male|    No|Sun|Dinner|   2|
|     10.33|1.67|Female|    No|Sun|Dinner|   3|
|     16.29|3.71|  Male|    No|Sun|Dinne

In [4]:
df_tips.printSchema()

root
 |-- total_bill: double (nullable = true)
 |-- tip: double (nullable = true)
 |-- sex: string (nullable = true)
 |-- smoker: string (nullable = true)
 |-- day: string (nullable = true)
 |-- time: string (nullable = true)
 |-- size: integer (nullable = true)



In [5]:
df_tips.columns

['total_bill', 'tip', 'sex', 'smoker', 'day', 'time', 'size']

In [6]:
### Handling Categorical Feature - Ordinal Encoding as all the variables are ordinal in nature
#Using StringIndexer

from pyspark.ml.feature import StringIndexer


In [7]:
indexer = StringIndexer(inputCols = ["sex","smoker","day","time"],outputCols = ["sex_indexed","smoker_indexed","day_indexed","time_indexed"])
df_tips_indexed = indexer.fit(df_tips).transform(df_tips)
df_tips_indexed.show()

+----------+----+------+------+---+------+----+-----------+--------------+-----------+------------+
|total_bill| tip|   sex|smoker|day|  time|size|sex_indexed|smoker_indexed|day_indexed|time_indexed|
+----------+----+------+------+---+------+----+-----------+--------------+-----------+------------+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|        1.0|           0.0|        1.0|         0.0|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|        0.0|           0.0|        1.0|         0.0|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|        0.0|           0.0|        1.0|         0.0|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2|        0.0|           0.0|        1.0|         0.0|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|        1.0|           0.0|        1.0|         0.0|
|     25.29|4.71|  Male|    No|Sun|Dinner|   4|        0.0|           0.0|        1.0|         0.0|
|      8.77| 2.0|  Male|    No|Sun|Dinner|   2|        0.0|           0.0|        1.0|         0.0|


In [8]:
df_tips_indexed.columns

['total_bill',
 'tip',
 'sex',
 'smoker',
 'day',
 'time',
 'size',
 'sex_indexed',
 'smoker_indexed',
 'day_indexed',
 'time_indexed']

In [9]:
from pyspark.ml.feature import VectorAssembler
featureassembler = VectorAssembler(inputCols = ["tip","size","sex_indexed","smoker_indexed","day_indexed","time_indexed"],outputCol="Independent Features")
output = featureassembler.transform(df_tips_indexed)

In [10]:
output.select("Independent Features").show()

+--------------------+
|Independent Features|
+--------------------+
|[1.01,2.0,1.0,0.0...|
|[1.66,3.0,0.0,0.0...|
|[3.5,3.0,0.0,0.0,...|
|[3.31,2.0,0.0,0.0...|
|[3.61,4.0,1.0,0.0...|
|[4.71,4.0,0.0,0.0...|
|[2.0,2.0,0.0,0.0,...|
|[3.12,4.0,0.0,0.0...|
|[1.96,2.0,0.0,0.0...|
|[3.23,2.0,0.0,0.0...|
|[1.71,2.0,0.0,0.0...|
|[5.0,4.0,1.0,0.0,...|
|[1.57,2.0,0.0,0.0...|
|[3.0,4.0,0.0,0.0,...|
|[3.02,2.0,1.0,0.0...|
|[3.92,2.0,0.0,0.0...|
|[1.67,3.0,1.0,0.0...|
|[3.71,3.0,0.0,0.0...|
|[3.5,3.0,1.0,0.0,...|
|(6,[0,1],[3.35,3.0])|
+--------------------+
only showing top 20 rows



In [11]:
final_data = output.select(["Independent Features","total_bill"])

In [12]:
final_data.show()

+--------------------+----------+
|Independent Features|total_bill|
+--------------------+----------+
|[1.01,2.0,1.0,0.0...|     16.99|
|[1.66,3.0,0.0,0.0...|     10.34|
|[3.5,3.0,0.0,0.0,...|     21.01|
|[3.31,2.0,0.0,0.0...|     23.68|
|[3.61,4.0,1.0,0.0...|     24.59|
|[4.71,4.0,0.0,0.0...|     25.29|
|[2.0,2.0,0.0,0.0,...|      8.77|
|[3.12,4.0,0.0,0.0...|     26.88|
|[1.96,2.0,0.0,0.0...|     15.04|
|[3.23,2.0,0.0,0.0...|     14.78|
|[1.71,2.0,0.0,0.0...|     10.27|
|[5.0,4.0,1.0,0.0,...|     35.26|
|[1.57,2.0,0.0,0.0...|     15.42|
|[3.0,4.0,0.0,0.0,...|     18.43|
|[3.02,2.0,1.0,0.0...|     14.83|
|[3.92,2.0,0.0,0.0...|     21.58|
|[1.67,3.0,1.0,0.0...|     10.33|
|[3.71,3.0,0.0,0.0...|     16.29|
|[3.5,3.0,1.0,0.0,...|     16.97|
|(6,[0,1],[3.35,3.0])|     20.65|
+--------------------+----------+
only showing top 20 rows



In [18]:
### Multiple Linear Regression using PySpark MLib

from pyspark.ml.regression import LinearRegression
train_data, test_data=final_data.randomSplit([0.75,0.25])
regressor = LinearRegression(featuresCol="Independent Features",labelCol="total_bill")
Model=regressor.fit(train_data)

In [19]:
Model.coefficients

DenseVector([3.1374, 3.5515, -0.7772, 1.8974, 0.1112, -1.3203])

In [20]:
Model.intercept

0.9428521938942336

In [21]:
##Predictions
prediction_results= Model.evaluate(test_data)
prediction_results.predictions.show()

+--------------------+----------+------------------+
|Independent Features|total_bill|        prediction|
+--------------------+----------+------------------+
|(6,[0,1],[1.25,2.0])|     10.07|11.967733085429815|
|(6,[0,1],[1.25,2.0])|     10.51|11.967733085429815|
| (6,[0,1],[2.0,3.0])|     16.31|17.872354480532046|
|(6,[0,1],[2.01,2.0])|     20.23|14.352193257383203|
| (6,[0,1],[2.5,4.0])|     18.35|22.992613976965394|
|(6,[0,1],[2.72,2.0])|     13.28|16.579781049602815|
| (6,[0,1],[3.0,2.0])|      14.0|17.458266376111954|
|(6,[0,1],[3.15,3.0])|     20.08| 21.48041921440888|
| (6,[0,1],[3.6,3.0])|     24.06| 22.89227063201286|
| (6,[0,1],[4.3,2.0])|      21.7|21.536948249190118|
| (6,[0,1],[5.0,3.0])|     31.27|27.284697264558574|
|[1.1,2.0,1.0,1.0,...|      12.9|12.617261863525517|
|[1.17,2.0,0.0,1.0...|     32.83|13.614126512552362|
|[1.44,2.0,0.0,1.0...|      7.74| 14.46123736311475|
|[1.5,2.0,1.0,0.0,...|     26.41|11.974851666699132|
|[1.58,2.0,0.0,1.0...|     13.42|13.9138379267

In [22]:
#Model Evaluation Metrics: Mean Absolute Error, Mean Squared Error
prediction_results.r2,prediction_results.meanAbsoluteError,prediction_results.meanSquaredError

(0.46426231379875094, 4.004589424612893, 35.34687238984888)